In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import math
import logging
logging.basicConfig(level=logging.ERROR)

import matplotlib.pyplot as plt

import os 
# import pandas

In [ ]:

#Network parameters
n_input = 12
n_hidden1 = 10
n_hidden2 = 10
n_output = 2

#Learning parameters
learning_constant = 0.2
number_epochs = 1000
batch_size = 1000

#Defining the input and the output
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])

#DEFINING WEIGHTS AND BIASES
b1 = tf.Variable(tf.random_normal([n_hidden1]))
b2 = tf.Variable(tf.random_normal([n_hidden2]))
b3 = tf.Variable(tf.random_normal([n_output]))
w1 = tf.Variable(tf.random_normal([n_input, n_hidden1]))
w2 = tf.Variable(tf.random_normal([n_hidden1, n_hidden2]))
w3 = tf.Variable(tf.random_normal([n_hidden2, n_output]))

def multilayer_perceptron(input_d):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(input_d, w1), b1))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, w2), b2))
    out_layer = tf.add(tf.matmul(layer_2, w3),b3)
    return out_layer


In [11]:
def loadHeartFailureDataset():    

    path = os.path.join("datasets","heart_failure_clinical_records_dataset.csv")
    data = np.genfromtxt(path, delimiter=",", names=True)

    data = data.view(np.float64).reshape((len(data), -1))
    x_data = data[:, 0:-1]
    y_data = data[:, -1]
    y_data = y_data.astype('int32')
    y_data = np.identity(2)[y_data] # one hot encoding

    # print(x_data)
    # print(y_data)
    return x_data, y_data

def splitLabel(data, number_of_labels):
    number_of_columns = len(data[0])

    x_data = data[:, 0:-number_of_labels]
    y_data = data[:, number_of_columns - number_of_labels : number_of_columns]
    return x_data, y_data

def mergeLabel(x_data, y_data):
    return np.column_stack((x_data, y_data))

def validationSplit(data, percentage):
    portion = round(len(data)*percentage)
    train = data[0:portion]
    test = data[portion:len(data)]
    return train, test

def minMaxNorm(data):
    spacing = 5
    pct_min = np.percentile(data, spacing, axis=0)
    pct_max = np.percentile(data, 100 - spacing, axis=0)
    norm_data = np.zeros(data.shape)
    for i in range(len(data[0])):
        norm_data[:, i] = (data[:,i] - pct_min[i]) / (pct_max[i] - pct_min[i])

    norm_data[norm_data > 1] = 1
    norm_data[norm_data < 0] = 0

    return norm_data


x_data, y_data = loadHeartFailureDataset()
norm_x_data = minMaxNorm(x_data)


i:  0 
5:  42.900000000000006 
95:  82.0
[75. 55. 65. 50. 65.]
[0.82097187 0.30946292 0.56521739 0.18158568 0.56521739]

i:  1 
5:  0.0 
95:  1.0
[0. 0. 0. 1. 1.]
[0. 0. 0. 1. 1.]

i:  2 
5:  59.0 
95:  2262.9999999999995
[ 582. 7861.  146.  111.  160.]
[0.23729583 3.5399274  0.03947368 0.02359347 0.04582577]

i:  3 
5:  0.0 
95:  1.0
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]

i:  4 
5:  20.0 
95:  60.0
[20. 38. 20. 20. 20.]
[0.   0.45 0.   0.   0.  ]

i:  5 
5:  0.0 
95:  1.0
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]

i:  6 
5:  131800.0 
95:  422499.9999999998
[265000.   263358.03 162000.   210000.   327000.  ]
[0.45820433 0.452556   0.10388717 0.26900585 0.67148263]

i:  7 
5:  0.7 
95:  3.0
[1.9 1.1 1.3 1.9 2.7]
[0.52173913 0.17391304 0.26086957 0.52173913 0.86956522]

i:  8 
5:  130.0 
95:  144.0
[130. 136. 129. 137. 116.]
[ 0.          0.42857143 -0.07142857  0.5        -1.        ]

i:  9 
5:  0.0 
95:  1.0
[1. 1. 1. 1. 0.]
[1. 1. 1. 1. 0.]

i:  10 
5:  0.0 
95:  1.0
[0. 0. 1. 0. 0.]
[0. 0. 1.

In [ ]:

#Create model
neural_network = multilayer_perceptron(X)

#Define loss and optimizer
loss_op = tf.reduce_mean(tf.math.squared_difference(neural_network,Y))
#loss_op =
tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=neural_network,labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_constant).minimize(loss_op)

#Initializing the variables
init = tf.global_variables_initializer()

x_data, y_data = loadHeartFailureDataset()
number_of_labels = len(y_data[0])

data = mergeLabel(x_data, y_data)
train_set, test_set = validationSplit(data, 0.8)
train_x, train_y = splitLabel(train_set, number_of_labels)
test_x, test_y = splitLabel(test_set, number_of_labels)

# batch_x1=np.loadtxt('x1.txt')
# batch_x2=np.loadtxt('x2.txt')
# batch_y1=np.loadtxt('y1.txt')
# batch_y2=np.loadtxt('y2.txt')

# label=batch_y2#+1e-50-1e-50

# batch_x=np.column_stack((batch_x1, batch_x2))
# batch_y=np.column_stack((batch_y1, batch_y2))

# batch_x_train=batch_x[:,0:599]
# batch_y_train=batch_y[:,0:599]
# batch_x_test=batch_x[:,600:1000]
# batch_y_test=batch_y[:,600:1000]

with tf.Session() as sess:
    sess.run(init)
    #Training epoch
    for epoch in range(number_epochs):
        sess.run(optimizer, feed_dict={X: train_x, Y: train_y})
        #Display the epoch
        if epoch % 100 == 0:
            print("Epoch:", '%d' % (epoch))

    # Test model
    pred = (neural_network) # Apply softmax to logits
    accuracy = tf.keras.losses.MSE(pred,Y)
    print("Accuracy:", accuracy.eval({X: train_x, Y: train_y}))
    
    #tf.keras.evaluate(pred,batch_x)
    print("Prediction:", pred.eval({X: train_x}))
    output=neural_network.eval({X: train_x})
    plt.plot(train_y[0:10], 'ro', output[0:10], 'bo')
    plt.ylabel('some numbers')
    plt.show()

    # estimated_class=tf.argmax(pred, 1)#+1e-50-1e-50
    # correct_prediction1 = tf.equal(tf.argmax(pred, 1),label)
    # accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1, tf.float32))
    
    # print(accuracy1.eval({X: batch_x}))